In [2]:
import cv2
import numpy as np
import time

In [4]:
def convert_to_grayscale(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return gray_frame

def display_green_objects(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_green = np.array([71, 77, 35])
    upper_green = np.array([98, 255, 190])

    green_mask = cv2.inRange(hsv_frame, lower_green, upper_green)

    kernel = np.ones((15, 15), np.uint8)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)

    combined_frame = np.vstack((frame, np.repeat(green_mask[:, :, np.newaxis], 3, axis=2)))
    return combined_frame

In [6]:
class FPS_Counter:
    def __init__(self, calc_time_perion_N_frames: int) -> None:
        self.time_buffer = []
        self.calc_time_perion_N_frames = calc_time_perion_N_frames

    def calc_FPS(self) -> float:
        time_buffer_is_full = len(self.time_buffer) == self.calc_time_perion_N_frames
        t = time.time()
        self.time_buffer.append(t)

        if time_buffer_is_full:
            self.time_buffer.pop(0)
            fps = len(self.time_buffer) / (self.time_buffer[-1] - self.time_buffer[0])
            return np.round(fps, 2)
        else:
            return 0.0

def calc_and_show_fps(frame, fps_counter):
    fps_real = fps_counter.calc_FPS()
    text = f"FPS: {fps_real:.1f}"

    fontFace = 1
    fontScale = 1.3
    thickness = 1
    
    (label_width, label_height), _ = cv2.getTextSize(
        text,
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
    )
    frame = cv2.rectangle(frame, (0, 0), (10 + label_width, 15 + label_height), (0, 0, 0), -1)
    frame = cv2.putText(
        img=frame,
        text=text,
        org=(5, 20),
        fontFace=fontFace,
        fontScale=fontScale,
        thickness=thickness,
        color=(255, 255, 255),
    )
    return frame

In [8]:
fps_counter = FPS_Counter(calc_time_perion_N_frames=15)

webcam = cv2.VideoCapture(0)

if not webcam.isOpened():
    print('Cant open webcam')

webcam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
webcam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = webcam.read()

    if not ret:
        print('Cant get frame from webcam')
        break

    #frame = convert_to_grayscale(frame)
    frame = display_green_objects(frame)

    scale = 0.5
    frame = cv2.resize(frame, (-1, -1), fx=scale, fy=scale)

    frame = calc_and_show_fps(frame, fps_counter)

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows()

2025-09-06 11:29:27.031 python[44844:3238915] +[IMKClient subclass]: chose IMKClient_Modern
2025-09-06 11:29:27.031 python[44844:3238915] +[IMKInputSession subclass]: chose IMKInputSession_Modern
